### This notebook will be an initial exploration of Folium's capabilities. 

Per the use cases supplied by ACEP, we want to create a map of Alaska with icons of solar installations over the various regions.  When the cursor hovers over the region, a box will pop up which displays the number of installations that were analyzed in that region along with the highest normalized annual production, and the average normalized annual production.  

In [1]:
import os
import folium
import pandas as pd

In [56]:
alaska_map = folium.Map(location = [63.977161, -152.024667], zoom_start = 4)
alaska_map

In [57]:
alaska_terrain = folium.Map(location = [63.977161, -152.024667], tiles = 'Stamen Terrain', zoom_start = 4)
alaska_terrain

In [58]:
tooltip = 'Click for more info'
#add a marker for the site of Fairbanks. Also, add a hoverover text box.
icon_type = 'star'
folium.Marker([64.838033, -147.668970], popup = 'Fairbanks, AK.', icon = folium.Icon(icon = icon_type),
              tooltip = tooltip).add_to(alaska_terrain)
alaska_terrain

In [59]:
#Add markers for other major cities.
#Anchorage
folium.Marker([61.193625, -149.694974], popup = 'Anchorage, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Kodiak
folium.Marker([57.794934, -152.392466], popup = 'Kodiak, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Nome
folium.Marker([64.516804, -165.399455], popup = 'Nome, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Juneau
folium.Marker([58.306096, -134.422802], popup = 'Juneau, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Prudhoe Bay
folium.Marker([70.212820, -148.408886], popup = 'Prudhoe Bay, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Kotzebue
folium.Marker([66.888948, -162.512280], popup = 'Kotzebue, AK.',
              tooltip = tooltip).add_to(alaska_terrain)
#Bethel
folium.Marker([60.794938, -161.770716], popup = 'Bethel, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Function that enables the map to show a lattitude and longitude when clicked.
alaska_terrain.add_child(folium.LatLngPopup())
#Display the map we've made.
alaska_terrain

You can also embed into each of these markers things like graphs. We would just need three lines of text, which would probably have to be hand-updated. Or, it could reference a variable which is set by a function call? Unclear what the right strategy is now, but we can definitely find an answer here. 

In [8]:
#To save this graphic as an interactive map:
#alaska_terrain.save("Practice Marker Map.html")

For us, the next step then would be to find a way to express data in these popups. I'll just do it for Fairbanks, as right now the S.o.A. Fairbanks panels are the ones we've been working with. 

In [63]:
#First step is to write the code that pulls in the data. We'll work with pre-formatted data. 
panel_data = pd.read_excel(io = 'spirit_of_alaska_fairbanks_cleaned.xlsx')
panel_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'spirit_of_alaska_fairbanks_cleaned.xlsx'

In [64]:
#Next, we'll need to generate a representation of the data we want to present as a table, in HTML
#First step: Calculate the normalized annual production. 
normalized_annual_production = panel_data.mean(axis = 0, skipna = True)
print(normalized_annual_production)
print(normalized_annual_production[1])
#Ok, and the last thing is to normalize by the capacity of the Johansen Solar array - 2.8 kW
johansen_normalized = normalized_annual_production[1]/2.8
print(johansen_normalized)

NameError: name 'panel_data' is not defined

In [65]:
#Ok, so we can also use PVWatts to calculate what a similar system should produce in a year.
#To make it comparable, we'll normalize the predictions based on the installation capacity.
#We'll simulate a 4 kW system, using normal pvwatts parameters and for Fairbanks AK.
#Our results is 3,698 kWh per year for a 4 kW DC system.
pvwatts = 3698
pvwatts_normalized = pvwatts/4

#Next, we'll make an entry for the number of installations: 1
number_of_installations = 1

#And finally, make the dataframe we'll use:
fairbanks_display_dataframe = pd.DataFrame(data = [['Number of Installations', int(number_of_installations)],
                                                  ['PVWatts Predicted Annual Power', pvwatts_normalized],
                                                  ['Average Produced Annual Power', johansen_normalized],
                                                  ['Maximum Produced Annual Power', johansen_normalized]],
                                           columns = ['Local Values', 'Quantity'])
print(fairbanks_display_dataframe)

NameError: name 'johansen_normalized' is not defined

In [66]:
#Ok, and now it's time to embed that dataframe!
html = fairbanks_display_dataframe.to_html()
fairbanks_popup = folium.Popup(html)
folium.Marker([64.838033, -147.668970], popup = fairbanks_popup).add_to(alaska_terrain)
alaska_terrain

NameError: name 'fairbanks_display_dataframe' is not defined

In [53]:
#And then save this graphic too. 
#alaska_terrain.save(os.path.join('alaska map.html', 'html_popups.html'))

In [67]:
#Exploring making the map icon interactive:
#Pulled directly offline - 
import folium
from ipywidgets import interact

cities = ["Berlin", "Paris", "Chicago", "Singapore"]
examples = ["Traffic", "Truck info", "Transit", "Regular", "Satellite"]
@interact(city=cities, example=examples)
def show_canned_examples(city, example):
    attr = "HERE.com"
    latlon_for_city = {
        "Berlin": (52.518611, 13.408333), 
        "Paris": (48.8567, 2.3508), 
        "Chicago": (41.88416, -87.63243),
        "Singapore": (1.283333, 103.833333)
    }
    zoom = 14
    queries = {
        "Traffic":
            "https://1.traffic.maps.api.here.com/maptile/2.1/traffictile/newest/normal.traffic.day/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Regular":
            "https://1.base.maps.api.here.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Truck info":
            "https://1.base.maps.api.here.com/maptile/2.1/trucktile/newest/normal.day.grey/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Transit":
            "https://1.base.maps.api.here.com/maptile/2.1/maptile/newest/normal.day.transit/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Satellite":
            "https://1.aerial.maps.api.here.com/maptile/2.1/maptile/newest/hybrid.day/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
    }
    return folium.Map(location=latlon_for_city[city], tiles=queries[example],attr=attr, zoom_start=zoom)

interactive(children=(Dropdown(description='city', options=('Berlin', 'Paris', 'Chicago', 'Singapore'), value=…

In [68]:
from ipywidgets import interact

cities = ["Berlin", "Paris", "Chicago", "Singapore"]
#examples = ["Traffic", "Truck info", "Transit", "Regular", "Satellite"]
@interact(city=cities)
def show_canned_examples(city):
    attr = "HERE.com"
    latlon_for_city = {
        "Berlin": (52.518611, 13.408333), 
        "Paris": (48.8567, 2.3508), 
        "Chicago": (41.88416, -87.63243),
        "Singapore": (1.283333, 103.833333)
    }
    zoom = 14
    return folium.Map(location = latlon_for_city[city])

interactive(children=(Dropdown(description='city', options=('Berlin', 'Paris', 'Chicago', 'Singapore'), value=…

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
def f(x):
    return x

In [4]:
#This is the slider bar code exactly that we would use for the tilt angle graph. 
interact(f, x = widgets.IntSlider(min = 5, max = 80, step = 5, value = 45));

interactive(children=(IntSlider(value=45, description='x', max=80, min=5, step=5), Output()), _dom_classes=('w…

In [45]:
import vincent
import os
import json
import requests
import numpy as np
import pandas as pd
vincent.core.initialize_notebook()

In [12]:
tilt = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
nrel_list_of_responses = []
for i in range(len(tilt)):
    list_parameters = {"format": 'JSON', "api_key": "DEMO_KEY", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": tilt[i], "azimuth": 180, "lat": 64.82, "lon": -147.87, "dataset": 'tmy2'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    new_dataframe = pd.DataFrame(data = json_response['outputs'])
    nrel_list_of_responses.append(new_dataframe)
print(nrel_list_of_responses[2])

    ac_monthly  poa_monthly  solrad_monthly  dc_monthly    ac_annual  \
0    35.106636    10.416152        0.336005   39.277702  3552.159912   
1   128.644897    36.141415        1.290765  136.692993  3552.159912   
2   333.818604    95.158379        3.069625  350.102051  3552.159912   
3   509.691620   148.750122        4.958337  533.102478  3552.159912   
4   545.316101   169.090500        5.454532  571.601929  3552.159912   
5   557.807922   176.559860        5.885329  585.247070  3552.159912   
6   532.229980   170.415588        5.497277  558.524292  3552.159912   
7   421.106140   132.087845        4.260898  442.421967  3552.159912   
8   269.292206    82.576073        2.752536  284.029602  3552.159912   
9   149.359085    43.960632        1.418085  159.239594  3552.159912   
10   55.902615    16.251221        0.541707   61.104649  3552.159912   
11   13.884660     4.638710        0.149636   16.694914  3552.159912   

    solrad_annual  capacity_factor  
0        2.967894        1

In [48]:
def disp_tilt_monthly(tilt_angle):
    index = tilt.index(tilt_angle)
    tilt_line_graph = vincent.Line(nrel_list_of_responses[index]['ac_monthly'])
    tilt_line_graph.axis_titles(x = "Month", y = "Energy Produced (kWh)")
    tilt_line_graph.display()

In [53]:
disp_tilt_monthly(45)

In [55]:
interact(disp_tilt_monthly, tilt_angle = widgets.IntSlider(min = 5, max = 80, step = 5, value = 45));

interactive(children=(IntSlider(value=45, description='tilt_angle', max=80, min=5, step=5), Output()), _dom_cl…

In [44]:
toy_list = [1,5,20,51]

line = vincent.Line(toy_list)
line.axis_titles(x = "index", y = "values")

In [69]:
lat, lon = input("Lat lon: ").split(',')

Lat lon: 23, 5


In [70]:
print(lat)

23


In [71]:
print(lon)

 5


In [76]:
type(int(lat))

int

In [ ]:
#Homer, with an interactive graph maybe
folium.Marker([59.645449, -151.543460], popup = , 
              tooltip = "Homer, AK").add_to(alaska_terrain)